In [239]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale,Normalizer,OneHotEncoder
from sklearn.preprocessing import normalize
from keras.models import Sequential,load_model
from tensorflow.python.keras.saving.saved_model import load as saved_model_load
from keras.layers import Dense, Dropout, Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
import keras.optimizers 

In [240]:
train = pd.read_csv('task3.training.csv')

In [241]:
test = pd.read_csv('task3.test.WITHOUT.answers.csv')

In [242]:
train_answer = train['class']

In [243]:
train['class']

0       B
1       J
2       H
3       G
4       G
       ..
9995    F
9996    B
9997    E
9998    H
9999    I
Name: class, Length: 10000, dtype: object

In [244]:
train = train.drop(columns = 'class')

In [245]:
train = np.array(train,dtype=np.float32)
test = np.array(test,dtype=np.float32)

In [246]:
meanlist=[]
stdlist =[]

In [247]:
# for i in range(np.shape(train)[1]):
#     tmp = np.sum(train[:,i])
#     mean = tmp/np.shape(train)[0]
#     meanlist.append(mean)
#     std = np.std(train[:,i])
#     stdlist.append(std)
#     for j in range(np.shape(train)[0]):
#         train[j][i] = (train[j][i]-mean)/std

In [248]:
len(meanlist)

0

In [249]:
len(stdlist)

0

In [250]:
# normalizer = Normalizer().fit(train) 
# train = normalizer.transform(train)

In [251]:
np.shape(train[:,1])

(10000,)

In [252]:
ans = pd.get_dummies(train_answer)

In [253]:
ans = np.array(ans)

In [254]:
train_feature_len = np.shape(train)[1]

In [255]:
train_x,val_x,train_y,val_y = train_test_split(train,ans,test_size=0.3,random_state=44)

In [256]:
np.shape(train_x)

(7000, 23)

In [257]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [263]:
lr= 1e-2
opti =keras.optimizers.Adam(learning_rate=lr)
model3 = Sequential()  
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*2, activation = 'relu',use_bias=True))
model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*2, activation = 'relu',use_bias=True))
model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*2, activation = 'relu',use_bias=True))
model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*0.5, activation = 'relu',use_bias=True))
model3.add(BatchNormalization())
model3.add(Dense(10, activation = 'softmax',use_bias=True))
para3 = model3.compile(optimizer = opti, loss = 'categorical_crossentropy', metrics = ['acc',f1_m,precision_m, recall_m])


In [264]:
# checkpoint
filepath="weights_with_normalize.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,
mode='max')
callbacks_list = [checkpoint]

In [265]:
#model3 = load_model('weights.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})
# model3 = load_model('weights_with_normalize.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})


In [266]:
# print('Training ------------')
# model3.fit(train_x, train_y, validation_split=0.33, epochs=1500,batch_size=128,callbacks=callbacks_list,verbose=0) #2000

In [308]:
print('Training ------------')
model3.fit(train_x, train_y,  epochs=1000,batch_size=128)

Training ------------
Epoch 1/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0257 - acc: 0.9919 - f1_m: 0.9919 - precision_m: 0.9919 - recall_m: 0.9919
Epoch 2/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0272 - acc: 0.9891 - f1_m: 0.9891 - precision_m: 0.9891 - recall_m: 0.9891
Epoch 3/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0166 - acc: 0.9940 - f1_m: 0.9940 - precision_m: 0.9940 - recall_m: 0.9940
Epoch 4/1000
55/55 [==============================] - 0s 3ms/step - loss: 0.0225 - acc: 0.9917 - f1_m: 0.9918 - precision_m: 0.9918 - recall_m: 0.9918
Epoch 5/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0403 - acc: 0.9846 - f1_m: 0.9847 - precision_m: 0.9847 - recall_m: 0.9847
Epoch 6/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0209 - acc: 0.9921 - f1_m: 0.9921 - precision_m: 0.9921 - recall_m: 0.9921
Epoch 7/1000
55/55 [==============================] - 0s 2ms/step - loss

55/55 [==============================] - 0s 2ms/step - loss: 0.0521 - acc: 0.9800 - f1_m: 0.9799 - precision_m: 0.9799 - recall_m: 0.9799
Epoch 55/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0270 - acc: 0.9900 - f1_m: 0.9900 - precision_m: 0.9900 - recall_m: 0.9900
Epoch 56/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0289 - acc: 0.9886 - f1_m: 0.9883 - precision_m: 0.9883 - recall_m: 0.9883
Epoch 57/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0224 - acc: 0.9921 - f1_m: 0.9921 - precision_m: 0.9922 - recall_m: 0.9920
Epoch 58/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0188 - acc: 0.9933 - f1_m: 0.9933 - precision_m: 0.9933 - recall_m: 0.9933
Epoch 59/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0299 - acc: 0.9894 - f1_m: 0.9894 - precision_m: 0.9894 - recall_m: 0.9894
Epoch 60/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0272 - acc: 0.9890 - f1_m

55/55 [==============================] - 0s 2ms/step - loss: 0.0381 - acc: 0.9871 - f1_m: 0.9872 - precision_m: 0.9872 - recall_m: 0.9872
Epoch 108/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0192 - acc: 0.9936 - f1_m: 0.9935 - precision_m: 0.9935 - recall_m: 0.9935
Epoch 109/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0231 - acc: 0.9911 - f1_m: 0.9908 - precision_m: 0.9908 - recall_m: 0.9908
Epoch 110/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0282 - acc: 0.9896 - f1_m: 0.9895 - precision_m: 0.9895 - recall_m: 0.9895
Epoch 111/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0282 - acc: 0.9900 - f1_m: 0.9900 - precision_m: 0.9900 - recall_m: 0.9900
Epoch 112/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0282 - acc: 0.9883 - f1_m: 0.9884 - precision_m: 0.9884 - recall_m: 0.9884
Epoch 113/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0415 - acc: 0.9859 

55/55 [==============================] - 0s 2ms/step - loss: 0.0211 - acc: 0.9917 - f1_m: 0.9917 - precision_m: 0.9917 - recall_m: 0.9917
Epoch 161/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0212 - acc: 0.9916 - f1_m: 0.9916 - precision_m: 0.9916 - recall_m: 0.9916
Epoch 162/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0189 - acc: 0.9924 - f1_m: 0.9922 - precision_m: 0.9922 - recall_m: 0.9922
Epoch 163/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0242 - acc: 0.9894 - f1_m: 0.9894 - precision_m: 0.9894 - recall_m: 0.9894
Epoch 164/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0145 - acc: 0.9947 - f1_m: 0.9947 - precision_m: 0.9947 - recall_m: 0.9947
Epoch 165/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0160 - acc: 0.9940 - f1_m: 0.9939 - precision_m: 0.9939 - recall_m: 0.9939
Epoch 166/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0280 - acc: 0.9891 

55/55 [==============================] - 0s 2ms/step - loss: 0.0132 - acc: 0.9954 - f1_m: 0.9953 - precision_m: 0.9953 - recall_m: 0.9953
Epoch 214/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0196 - acc: 0.9929 - f1_m: 0.9927 - precision_m: 0.9927 - recall_m: 0.9927
Epoch 215/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0208 - acc: 0.9931 - f1_m: 0.9932 - precision_m: 0.9932 - recall_m: 0.9932
Epoch 216/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0242 - acc: 0.9911 - f1_m: 0.9911 - precision_m: 0.9911 - recall_m: 0.9911
Epoch 217/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0407 - acc: 0.9859 - f1_m: 0.9859 - precision_m: 0.9859 - recall_m: 0.9859
Epoch 218/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0328 - acc: 0.9880 - f1_m: 0.9881 - precision_m: 0.9881 - recall_m: 0.9881
Epoch 219/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0180 - acc: 0.9934 

...

55/55 [==============================] - 0s 2ms/step - loss: 0.0600 - acc: 0.9793 - f1_m: 0.9793 - precision_m: 0.9793 - recall_m: 0.9793
Epoch 744/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0207 - acc: 0.9923 - f1_m: 0.9923 - precision_m: 0.9923 - recall_m: 0.9923
Epoch 745/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0282 - acc: 0.9900 - f1_m: 0.9899 - precision_m: 0.9899 - recall_m: 0.9899
Epoch 746/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0222 - acc: 0.9923 - f1_m: 0.9923 - precision_m: 0.9923 - recall_m: 0.9923
Epoch 747/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0203 - acc: 0.9939 - f1_m: 0.9939 - precision_m: 0.9939 - recall_m: 0.9939
Epoch 748/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0183 - acc: 0.9927 - f1_m: 0.9926 - precision_m: 0.9926 - recall_m: 0.9926
Epoch 749/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0165 - acc: 0.9934 

55/55 [==============================] - 0s 2ms/step - loss: 0.0262 - acc: 0.9891 - f1_m: 0.9890 - precision_m: 0.9890 - recall_m: 0.9890
Epoch 797/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0457 - acc: 0.9830 - f1_m: 0.9830 - precision_m: 0.9830 - recall_m: 0.9830
Epoch 798/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0247 - acc: 0.9906 - f1_m: 0.9906 - precision_m: 0.9906 - recall_m: 0.9906
Epoch 799/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0216 - acc: 0.9916 - f1_m: 0.9916 - precision_m: 0.9916 - recall_m: 0.9916
Epoch 800/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0296 - acc: 0.9891 - f1_m: 0.9891 - precision_m: 0.9891 - recall_m: 0.9890
Epoch 801/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0216 - acc: 0.9914 - f1_m: 0.9914 - precision_m: 0.9914 - recall_m: 0.9914
Epoch 802/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0185 - acc: 0.9927 

55/55 [==============================] - 0s 2ms/step - loss: 0.0211 - acc: 0.9924 - f1_m: 0.9922 - precision_m: 0.9922 - recall_m: 0.9922
Epoch 850/1000
55/55 [==============================] - 0s 3ms/step - loss: 0.0164 - acc: 0.9939 - f1_m: 0.9937 - precision_m: 0.9937 - recall_m: 0.9937
Epoch 851/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0163 - acc: 0.9949 - f1_m: 0.9949 - precision_m: 0.9949 - recall_m: 0.9949
Epoch 852/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0148 - acc: 0.9946 - f1_m: 0.9945 - precision_m: 0.9945 - recall_m: 0.9945
Epoch 853/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0160 - acc: 0.9940 - f1_m: 0.9940 - precision_m: 0.9940 - recall_m: 0.9940
Epoch 854/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0150 - acc: 0.9947 - f1_m: 0.9947 - precision_m: 0.9947 - recall_m: 0.9947
Epoch 855/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0162 - acc: 0.9950 

55/55 [==============================] - 0s 2ms/step - loss: 0.0177 - acc: 0.9930 - f1_m: 0.9930 - precision_m: 0.9930 - recall_m: 0.9930
Epoch 903/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0287 - acc: 0.9889 - f1_m: 0.9889 - precision_m: 0.9889 - recall_m: 0.9889
Epoch 904/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0215 - acc: 0.9921 - f1_m: 0.9921 - precision_m: 0.9921 - recall_m: 0.9921
Epoch 905/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0178 - acc: 0.9939 - f1_m: 0.9939 - precision_m: 0.9939 - recall_m: 0.9939
Epoch 906/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0300 - acc: 0.9886 - f1_m: 0.9886 - precision_m: 0.9886 - recall_m: 0.9886
Epoch 907/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0335 - acc: 0.9861 - f1_m: 0.9862 - precision_m: 0.9862 - recall_m: 0.9862
Epoch 908/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0233 - acc: 0.9906 

55/55 [==============================] - 0s 2ms/step - loss: 0.0153 - acc: 0.9939 - f1_m: 0.9940 - precision_m: 0.9941 - recall_m: 0.9938
Epoch 956/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0183 - acc: 0.9927 - f1_m: 0.9927 - precision_m: 0.9927 - recall_m: 0.9927
Epoch 957/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0306 - acc: 0.9896 - f1_m: 0.9896 - precision_m: 0.9896 - recall_m: 0.9896
Epoch 958/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0250 - acc: 0.9910 - f1_m: 0.9911 - precision_m: 0.9911 - recall_m: 0.9911
Epoch 959/1000
55/55 [==============================] - 0s 2ms/step - loss: 0.0295 - acc: 0.9896 - f1_m: 0.9896 - precision_m: 0.9896 - recall_m: 0.9896
Epoch 960/1000
55/55 [==============================] - 0s 3ms/step - loss: 0.0173 - acc: 0.9939 - f1_m: 0.9939 - precision_m: 0.9939 - recall_m: 0.9939
Epoch 961/1000
55/55 [==============================] - 0s 3ms/step - loss: 0.0388 - acc: 0.9867 

In [309]:
# model3.save('ML_task2.h5')

In [310]:
# # model3 = load_model('weights.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})
# model3 = load_model('weights_with_normalize.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})


In [311]:
print('\nTesting ------------')
loss, accuracy, f1_score, precision, recall = model3.evaluate(val_x, val_y)
print('\ntest loss: ', loss)
print('\ntest f1_score: ', f1_score) #no normal best 0.9746232628822327 


Testing ------------
94/94 [==============================] - 0s 953us/step - loss: 0.2659 - acc: 0.9360 - f1_m: 0.9361 - precision_m: 0.9361 - recall_m: 0.9361

test loss:  0.26593509316444397

test f1_score:  0.9360594153404236


In [312]:
test.shape

(2000, 23)

In [138]:
for i in range(np.shape(test)[1]):
    mean = meanlist[i]
    std  = stdlist[i]
    for j in range(np.shape(test)[0]):
        test[j][i] = (test[j][i]-mean)/std

In [81]:
np.sum(test[:,1])

20549.914

In [82]:
prediction = model3.predict(test)

In [83]:
np.argmax(prediction[0], axis=0)

0

In [84]:
alph = ['A','B','C','D','E','F','G','H','I','J']
prediction_final=[]
for i in range(np.shape(prediction)[0]):
    index = np.argmax(prediction[i], axis=0)
    prediction_final.append(alph[index])

In [29]:
# prediction_final

In [22]:
tt = dict([('id',np.array(range(1,2001))),('Predicted',prediction_final)])
tt = pd.DataFrame(tt)
tt.to_csv('test.csv',index=False)

NameError: name 'prediction_final' is not defined

In [23]:
## sklearn

In [32]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification

clf = MLPClassifier(random_state=1,hidden_layer_sizes=(train_feature_len,train_feature_len*2,train_feature_len*3,train_feature_len*2,train_feature_len,15),learning_rate_init=0.001, max_iter=500,learning_rate = 'adaptive').fit(train_x, train_y)
clf.predict_proba(val_x)

clf.predict(val_x)

clf.score(val_x,val_y)

0.945

In [25]:
for i in range(np.shape(test)[1]):
    mean = meanlist[i]
    std  = stdlist[i]
    for j in range(np.shape(test)[0]):
        test[j][i] = (test[j][i]-mean)/std

In [26]:
prediction = clf.predict(test)

In [27]:
alph = ['A','B','C','D','E','F','G','H','I','J']
prediction_final=[]
for i in range(np.shape(prediction)[0]):
    index = np.argmax(prediction[i], axis=0)
    prediction_final.append(alph[index])

In [28]:
tt = dict([('id',np.array(range(1,2001))),('Predicted',prediction_final)])
tt = pd.DataFrame(tt)
tt.to_csv('test.csv',index=False)